# How to contribute?

Here we report a simple guide with the key features of RobustX in order to implement your own (robust) counterfactual explanation method.

# Setup preparation

In [2]:
# Import necessary components
from sklearn.model_selection import train_test_split
from robustx.lib.models.pytorch_models.SimpleNNModel import SimpleNNModel
from robustx.datasets.ExampleDatasets import get_example_dataset
from robustx.lib.tasks.ClassificationTask import ClassificationTask
from robustx.generators.CE_methods.Wachter import Wachter
import numpy as np
from robustx.generators.CEGenerator import CEGenerator
from robustx.generators.robust_CE_methods.EntropicRiskCE import EntropicRiskCE
from robustx.lib.models.pytorch_models.EnsembleModelWrapper import EnsembleModelWrapper


In [3]:
# Load and preprocess dataset
dl = get_example_dataset("iris")
dl.preprocess(
    impute_strategy_numeric='mean',  # Impute missing numeric values with mean
    scale_method='minmax',           # Apply min-max scaling
    encode_categorical=False         # No categorical encoding needed (since no categorical features)
)

# remove the target column from the dataset that has labels 2
dl.data = dl.data[dl.data['target'] != 2]

# Load model, note some RecourseGenerators may only work with a certain type of model,
# e.g., MCE only works with a SimpleNNModel
n_models = 10
model_ensemble = [SimpleNNModel(4, [10], 1, seed=0) for _ in range(n_models)]

target_column = "target"
X_train, X_test, y_train, y_test = train_test_split(dl.data.drop(columns=[target_column]), dl.data[target_column], test_size=0.35, random_state=0)


# Train each model in the ensemble
all_indexes = np.arange(X_train.shape[0])
for model in model_ensemble:
    np.random.shuffle(all_indexes)
    sampled_indexes = all_indexes[:int(0.8 * len(all_indexes))]
    model.train(X_train.iloc[sampled_indexes], y_train.iloc[sampled_indexes], epochs=100, batch_size=16, verbose=0)
    print(f"model accuracy: {model.compute_accuracy(X_test.values, y_test.values):0.4f}")

emodel = EnsembleModelWrapper(model_ensemble=model_ensemble, aggregation_method='majority_vote')
print(f"ensemble accuracy: {emodel.compute_accuracy(X_test, y_test):0.4f}")


# Create task
task = ClassificationTask(emodel, dl)

model accuracy: 0.6000
model accuracy: 0.4286
model accuracy: 0.4571
model accuracy: 0.5143
model accuracy: 0.5143
model accuracy: 0.6000
model accuracy: 0.4571
model accuracy: 0.5143
model accuracy: 0.5143
model accuracy: 0.4571
ensemble accuracy: 0.5143


# Example of an already implemented CE generation method in RobustX

In [10]:
# Each counterfactual explanation generator takes the task on creation, it can also take a custom distance function, but for now we will use the default one.
ce_gen = EntropicRiskCE(task)
base_cf_gen_class = Wachter
base_cf_gen_args = {
    'target_class': 1,
    'max_iter': 100,
    'lr': 0.01,
    'lambda_param': 0.1,
    'device': 'cpu'
}

# Get negative instances, the default column_name is always "target" but you can set it to the name of your dataset's target variable
negs = dl.get_negative_instances(neg_value=0, column_name="target")
print("Negative instances shape: ", negs.shape)
print(f"Example of a prediction for a negative instance:\n")
print(negs.head(1))
print("Output: ", emodel.predict(negs.head(1)).values.item())
print("Class: ", int(emodel.predict(negs.head(1)).values.item() > 0.5))  # Assuming binary classification with threshold 0.5

# You can generate for a set of instances stored in a DataFrame
print("\nGenerating counterfactual explanations using EntropicRiskCF for the first 5 negative instances:")
ce = ce_gen.generate_for_instance(negs.iloc[0],
                                  base_cf_gen_class=base_cf_gen_class,
                                  base_cf_gen_args=base_cf_gen_args,
                                  verbose=True, 
                                  device='cpu')
print(ce)
print("Output: ", model.predict(ce).values.item())
print("Class: ", int(model.predict(ce).values.item() > 0.5))  # Assuming binary classification with threshold 0.5

Negative instances shape:  (50, 4)
Example of a prediction for a negative instance:

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0           0.222222             0.625           0.067797          0.041667
Output:  0
Class:  0

Generating counterfactual explanations using EntropicRiskCF for the first 5 negative instances:
Iteration 01: Entropic risk = 0.4961
Current CE: [[ 0.42861804  0.41806227  0.26978323 -0.16446689]]
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0           0.428618          0.418062           0.269783         -0.164467
Output:  0.5014529228210449
Class:  1


In [12]:
# You can also implement a method to generate CEs for all the negative instance in one shot
ces = ce_gen.generate_for_all(neg_value=0, 
                              column_name="target", 
                              base_cf_gen_class=base_cf_gen_class, 
                              base_cf_gen_args=base_cf_gen_args, 
                              device='cpu',
                              verbose=True)
print("All outputs are positive? ", np.all(model.predict(ces)>0.5))

Iteration 01: Entropic risk = 0.4963
Current CE: [[ 0.42900485  0.41951373  0.2673123  -0.16364896]]
Iteration 01: Entropic risk = 0.4924
Current CE: [[ 0.3747817   0.22446747  0.2561188  -0.1566531 ]]
Iteration 01: Entropic risk = 0.5032
Current CE: [[ 0.30012894  0.33530894  0.20943256 -0.13182944]]
Iteration 02: Entropic risk = 0.5016
Current CE: [[ 0.31012896  0.32530895  0.21943256 -0.14182945]]
Iteration 03: Entropic risk = 0.5000
Current CE: [[ 0.32012898  0.31530893  0.22943257 -0.15182945]]
Iteration 01: Entropic risk = 0.4975
Current CE: [[ 0.30108747  0.2555828   0.2783528  -0.16568267]]
Iteration 01: Entropic risk = 0.4926
Current CE: [[ 0.44122243  0.41845855  0.31355253 -0.19486569]]
Iteration 01: Entropic risk = 0.4928
Current CE: [[ 0.58408755  0.5069499   0.38709766 -0.15382746]]
Iteration 01: Entropic risk = 0.4985
Current CE: [[ 0.32913458  0.34143132  0.3034562  -0.1590659 ]]
Iteration 01: Entropic risk = 0.4922
Current CE: [[ 0.42075735  0.35832173  0.3053008  -0.1

# Benchmarking your method

After you have finished implementing your method, you can include it into DefaultBenchmark.py file and test it against other methods supported in the library using this lines of code:

In [14]:
from robustx.lib.DefaultBenchmark import default_benchmark
methods = ["KDTreeNNCE", "EntropicRiskCE"]
evaluations = ["Validity", "Distance"]
default_benchmark(task, methods, evaluations, 
                  neg_value=0, 
                  column_name="target", 
                  delta=0.005,
                  base_cf_gen_class=base_cf_gen_class,
                  base_cf_gen_args=base_cf_gen_args,
                  tau=0.8,
                  device='cpu')

+----------------+----------------------+------------+------------+
| Method         |   Execution Time (s) |   Validity |   Distance |
+================+======================+============+============+
| KDTreeNNCE     |             0.147826 |   1        |   0.447422 |
+----------------+----------------------+------------+------------+
| EntropicRiskCE |             0.654669 |   0.617284 |   0.270493 |
+----------------+----------------------+------------+------------+
